Below is a demo on how to use this brew recipe builder.

The first thing to do is find the ingredients you want to use in the recipe. You can search the tables in the database within the notebook using the code below. As a note, the tables you most likely want to search are: fermentable, hop, yeast and style.

In [1]:
from brew_builder import BrewBuild, search_db, menu_select
import numpy as np
import sqlite3

# connect to the database

con = sqlite3.connect('database/default_db.sqlite')

In [2]:
# search some table using connection to database
# this will display dataframe with search results
# you can type in the search box to change results
menu_select('fermentable', con)

interactive(children=(Text(value='Dry', description='fermentable:', placeholder='Type something'), Output()), …

When searching for ingredients, it will be important to make notes of the id of the ingredient. You will need this to build the recipe in the next step.

Once you find all of your ingredients, you can now build a recipe! This is done by initilzing the object below:

In [3]:
# set up the grain bill
# the format of each row is [id, amount (lbs), type (0=mash, 1=extract)]
grain_bill = np.array([[9, 0.25, 0],
                       [122, 1, 0],
                       [93, 1, 1],
                       [90, 1, 1],
                       [13, 1.5, 0],
                       [24, 0.5, 0]])

# set up the hop bill
# the format of each row is [id, amount (oz), boil time (min)]
hop_bill = np.array([[62, 1, 45],
                     [21, 0.5, 20]])

# yeast id
yeast = 3

# target volume in gallons
target_volume = 2.5

# boil volume in gallons
boil_volume = 1.75

# mash temp in F
mash_temp = 165

# boil time in min
boil_time = 45

# estimated mash efficiency
mash_efficiency = 48

# the style id
style = 64

# the mash volume in gallons
mash_volume = 1.5

build = BrewBuild(grain_bill, hop_bill, yeast, target_volume,
                               boil_volume, mash_temp, con, boil_time=boil_time,
                               mash_efficiency=mash_efficiency, style=style, mash_volume=mash_volume)

Once the object is setup, below are examples of some of the basic calculations you can do

In [4]:
build.calc_OG()

1.056

In [5]:
build.calc_FG()

1.017

In [6]:
build.calc_ABV(build.calc_OG(), build.calc_FG())

5.12

In [7]:
build.calc_color()

42.0

In [8]:
build.calc_IBU()

27.2

You can also then just build the recipe, which does all of the calculations and creates your recipe in a csv file.

In [9]:
build.build_recipe('Oatmeal_Stout_Test.csv')

You may not be happy with the initial recipe though. So, after you have built the recipe once, you can create an interactive spreadsheet in the notebook which allows you to edit the recipe and see the effects of the changes in real time. Currently, you cant change any of the ingredients in the cells, just their values. The way to think about this sheet, it you can really change anything in a yellow cell and everything in red will be recalculated.

In [10]:
sheet = build.interactive_sheet()

In [11]:
sheet

Sheet(cells=(Cell(column_end=0, column_start=0, row_end=0, row_start=0, type='text', value='Summary'), Cell(co…

After making these changes in the spreadsheet above, you can actually then rewrite your recipe based on the changes you have made! This is done below.

In [12]:
build.update_recipe_from_sheet(sheet, 'Oatmeal_Stout_Edited_Test.csv')

Once done, it is best to then close out the connection to the database and get brewing!

In [13]:
con.close()